In [11]:
# import all required labraries

import numpy as np
import pandas as pd
import tensorflow as tf
import os
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

In [12]:
# now define path to dataset
path="ImagePro"
files=os.listdir(path)
# list of files in path
# sort path from A-Y
files.sort()

In [13]:
# print to see list
print(files)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y']


In [14]:
# create list of image and label

image_array=[] # all images
label_array=[] # corresponding labels (0-24)


In [15]:
# loop through each file in files
for i in tqdm(range(len(files))):
	# list of image in each folder
	sub_file=os.listdir(path+"/"+files[i])
	# let's check length of each folder
	#	print(len(sub_file))

	# loop through each sub folder
	for j in range(len(sub_file)):

		# path of each image
		#Example:imagepro/A/image_name1.jpg

		file_path=path+"/"+files[i]+"/"+sub_file[j]
		# read each image

		image=cv2.imread(file_path)

		# resize image by 96x96
		image=cv2.resize(image,(96,96))
		# convert BGR image to RGB image
		image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)

		# add this image at image_array
		image_array.append(image)

		# add label to label_array
		# i is number from 0 to len(files)-1
		# so we can use it as label
		label_array.append(i)

100%|██████████| 25/25 [00:17<00:00,  1.45it/s]


In [16]:
# convert list to array
image_array=np.array(image_array)
label_array=np.array(label_array,dtype="float")

In [17]:
# split the dataset into test and train
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(image_array,label_array,test_size=0.15)

In [18]:
del image_array,label_array
# to free memory 
import gc
gc.collect()

967

In [19]:
# Create a model
from keras import layers,callbacks,utils,applications,optimizers
from keras.models import Sequential, Model, load_model

model=Sequential()

pretrained_model=tf.keras.applications.EfficientNetB0(input_shape=(96,96,3),include_top=False)
model.add(pretrained_model)

# add Pooling to model
model.add(layers.GlobalAveragePooling2D())

# add dropout to model
# We add dropout to increase accuracy by reduce overfitting
model.add(layers.Dropout(0.3))
# finally we will addd dense layer as an output
model.add(layers.Dense(1))
# For some tensorflow version we required to build model
model.build(input_shape=(None,96,96,3))

16705208/16705208 [==============================] - 3s 0us/step


In [20]:
# to see model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional)  (None, 3, 3, 1280)       4049571   
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 1)                 1281      
                                                                 
Total params: 4,050,852
Trainable params: 4,008,829
Non-trainable params: 42,023
_________________________________________________________________


In [21]:
# compile model
# you can use different optimizer and loss function to increase accuracy
model.compile(optimizer="adam",loss="mae",metrics=["mae"])

#mae-> Mean Absolute Error

# creating a checkpoint to save best accuracy model
ckp_path="trained_model/model"
model_checkpoint=tf.keras.callbacks.ModelCheckpoint(
									filepath=ckp_path,
									monitor="val_mae",
									mode="auto",
									save_best_only=True,
									save_weights_only=True
									)
# monitor: monitor validation mae loss to save model
# mode: Use to save model when val_mae is minimum or maximum
# It has 3 option: "min","max","auto".
# for us you can select either "min" or "auto"
# When val_mae reduce model will be saved
# save_best_only: False -> It will save all model
# save_weights_only: Save only weight.

In [23]:
# create learning rate reducer to reduce lr when accuracy does not improve
# Correct 
reduce_lr=tf.keras.callbacks.ReduceLROnPlateau(
									factor=0.9,
									monitor="val_mae",
									mode="auto",
									cooldown=0,
									patience=5,
									verbose=1,
									min_lr=1e-6)

# factor: when it is reduce next lr will be 0.9 times of current
# next lr= 0.9* current lr

# patience=X
# reduce lr after X epoch when accuracy does not improve
# verbose : show it after every epoch

# min_lr : minimum learning rate 

# Start training model

Epochs=5
Batch_Size=32

#X_train,X_test,Y_train,Y_test
history=model.fit(
				X_train,
				Y_train,
				validation_data=(X_test,Y_test),
				batch_size=Batch_Size,
				epochs=Epochs,
				callbacks=[model_checkpoint,reduce_lr]
				)

# Before training you can delete image_array and label_array to increase ram memory 

# after the training is done load best model

model.load_weights(ckp_path)





Epoch 1/5
303/303 [==============================] - 622s 2s/step - loss: 1.3153 - mae: 1.3153 - val_loss: 0.6986 - val_mae: 0.6986 - lr: 0.0010
Epoch 2/5
303/303 [==============================] - 543s 2s/step - loss: 0.9324 - mae: 0.9324 - val_loss: 0.9625 - val_mae: 0.9625 - lr: 0.0010
Epoch 3/5
303/303 [==============================] - ETA: 0s - loss: 0.7415 - mae: 0.7415

KeyboardInterrupt: 

In [24]:
# convert model to tensorflow lite model

converter=tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model=converter.convert()

# save model
with open("model.tflite","wb") as f:
	f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\arpit\AppData\Local\Temp\tmpza4gg_sm\assets


INFO:tensorflow:Assets written to: C:\Users\arpit\AppData\Local\Temp\tmpza4gg_sm\assets


In [25]:
# if you want to see prediction result on test dataset
prediction_val=model.predict(X_test,batch_size=32)

# print first 10 values
print(prediction_val[:10])
# print first 10 values of Y_test
print(Y_test[:10])

54/54 [==============================] - 27s 482ms/step
[[17.988003 ]
 [25.076118 ]
 [ 6.3167577]
 [ 4.939446 ]
 [ 6.0752187]
 [16.99824  ]
 [ 5.386086 ]
 [13.546445 ]
 [ 9.117499 ]
 [ 6.089885 ]]
[18. 24.  6.  5.  6. 16.  5. 13.  9.  6.]


In [28]:
print(model.accuracy())

76.942
